In [1]:
!pip install -q --upgrade nltk gensim bokeh pandas

import nltk
nltk.download('punkt')
nltk.download('stopwords')

You should consider upgrading via the '/usr/bin/python3 -m pip install --upgrade pip' command.


[nltk_data] Downloading package punkt to /home/mikhail/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /home/mikhail/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [2]:
import os
import pandas as pd
import numpy as np
from nltk.tokenize import word_tokenize
from gensim.models import Word2Vec
from sklearn.manifold import TSNE

/home/mikhail/.local/lib/python3.8/site-packages/gensim/similarities/__init__.py:15: UserWarning: The gensim.similarities.levenshtein submodule is disabled, because the optional Levenshtein package <https://pypi.org/project/python-Levenshtein/> is unavailable. Install Levenhstein (e.g. `pip install python-Levenshtein`) to suppress this warning.
  warnings.warn(msg)


In [3]:
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.corpus import stopwords
from string import punctuation

Пути к файлам.

In [4]:
ROOT_PATH = './data'

ru_vec_path = os.path.join(ROOT_PATH, 'cc.ru.300.vec')
ukr_vec_path = os.path.join(ROOT_PATH, 'cc.uk.300.vec')

train_path = os.path.join(ROOT_PATH, 'ukr_rus.train.txt')
test_path = os.path.join(ROOT_PATH, 'ukr_rus.test.txt')
fairy_tale_path = os.path.join(ROOT_PATH, 'fairy_tale.txt')

## Запилим пословный машинный перевод!

Напишем простенькую реализацию модели машинного перевода.

Идея основана на статье [Word Translation Without Parallel Data](https://arxiv.org/pdf/1710.04087.pdf). У авторов в репозитории еще много интересного: [https://github.com/facebookresearch/MUSE](https://github.com/facebookresearch/MUSE).

А мы будем переводить с украинского на русский.

![](https://raw.githubusercontent.com/yandexdataschool/nlp_course/master/resources/blue_cat_blue_whale.png)   
*синій кіт* vs. *синій кит*

In [5]:
from gensim.models import KeyedVectors

ru_emb = KeyedVectors.load_word2vec_format(ru_vec_path)
uk_emb = KeyedVectors.load_word2vec_format(ukr_vec_path)

Посмотрим на пару серпень-август (являющихся переводом)

In [6]:
ru_emb.most_similar([ru_emb["август"]])

[('август', 1.0000001192092896),
 ('июль', 0.9383152723312378),
 ('сентябрь', 0.9240029454231262),
 ('июнь', 0.9222574830055237),
 ('октябрь', 0.9095539450645447),
 ('ноябрь', 0.8930036425590515),
 ('апрель', 0.8729087114334106),
 ('декабрь', 0.8652557730674744),
 ('март', 0.8545795679092407),
 ('февраль', 0.8401415944099426)]

In [7]:
uk_emb.most_similar([uk_emb["серпень"]])

[('серпень', 0.9999998807907104),
 ('липень', 0.9096441268920898),
 ('вересень', 0.9016969203948975),
 ('червень', 0.8992518782615662),
 ('жовтень', 0.8810408115386963),
 ('листопад', 0.8787633180618286),
 ('квітень', 0.8592804670333862),
 ('грудень', 0.8586863279342651),
 ('травень', 0.840811014175415),
 ('лютий', 0.8256431221961975)]

In [8]:
ru_emb.most_similar([uk_emb["серпень"]])

[('Недопустимость', 0.24435284733772278),
 ('конструктивность', 0.23293082416057587),
 ('офор', 0.23256804049015045),
 ('deteydlya', 0.230317160487175),
 ('пресечении', 0.22632381319999695),
 ('одностороннего', 0.22608886659145355),
 ('подход', 0.2230587750673294),
 ('иболее', 0.22003726661205292),
 ('2015Александр', 0.21872766315937042),
 ('конструктивен', 0.21796567738056183)]

In [9]:
def load_word_pairs(filename):
    uk_ru_pairs = []
    uk_vectors = []
    ru_vectors = []
    with open(filename, "r", encoding='utf8') as inpf:
        for line in inpf:
            uk, ru = line.rstrip().split("\t")
            if uk not in uk_emb or ru not in ru_emb:
                continue
            uk_ru_pairs.append((uk, ru))
            uk_vectors.append(uk_emb[uk])
            ru_vectors.append(ru_emb[ru])
    return uk_ru_pairs, np.array(uk_vectors), np.array(ru_vectors)


uk_ru_train, X_train, Y_train = load_word_pairs(train_path)
uk_ru_test, X_test, Y_test = load_word_pairs(test_path)

### Учим маппинг из одного пространства эмбеддингов в другое

У нас есть пары слов, соответствующих друг другу, и их эмбеддинги. Найдем преобразование из одного пространства в другое, чтобы приблизить известные нам слова:

$$W^*= \arg\min_W ||WX - Y||_F, \text{где} ||*||_F - \text{норма Фробениуса}$$

Эта функция очень похожа на линейную регрессию (без биаса).

**Задание** Реализуйте её - воспользуйтесь `LinearRegression` из sklearn с `fit_intercept=False`:

In [10]:
from sklearn.linear_model import LinearRegression

mapping = LinearRegression(fit_intercept=False)

mapping.fit(X_train, Y_train)

LinearRegression(fit_intercept=False)

Проверим, куда перейдет `серпень`:

In [11]:
august = mapping.predict(uk_emb["серпень"].reshape(1, -1))
ru_emb.most_similar(august)

[('апрель', 0.8541286587715149),
 ('июнь', 0.8411202430725098),
 ('март', 0.839699387550354),
 ('сентябрь', 0.835986852645874),
 ('февраль', 0.8329297304153442),
 ('октябрь', 0.8311845660209656),
 ('ноябрь', 0.8278923034667969),
 ('июль', 0.8234528303146362),
 ('август', 0.8120499849319458),
 ('декабрь', 0.8039003610610962)]

Должно получиться, что в топе содержатся разные месяцы, но август не первый.

Будем мерять percision top-k с k = 1, 5, 10.

**Задание** Реализуйте следующую функцию:

In [12]:
def precision(pairs, mapped_vectors, topn=1):
    """
    :args:
        pairs = list of right word pairs [(uk_word_0, ru_word_0), ...]
        mapped_vectors = list of embeddings after mapping from source embedding space to destination embedding space
        topn = the number of nearest neighbours in destination embedding space to choose from
    :returns:
        precision_val, float number, total number of words for those we can find right translation at top K.
    """
    assert len(pairs) == len(mapped_vectors)
    num_matches = 0
    for i, (_, ru) in enumerate(pairs):
        # получение topn похожих слов 
        topn_similar = ru_emb.most_similar(mapped_vectors[i][np.newaxis, ...], topn=topn)
        # транспонирование массива и получение нулевого элемента (кортеж слов)
        topn_similar = list(zip(*topn_similar))[0]
        if ru in topn_similar:
            num_matches += 1
    precision_val = num_matches / len(pairs)
    return precision_val

In [13]:
assert precision([("серпень", "август")], august, topn=5) == 0.0
assert precision([("серпень", "август")], august, topn=9) == 1.0
assert precision([("серпень", "август")], august, topn=10) == 1.0

In [14]:
assert precision(uk_ru_test, X_test) == 0.0
assert precision(uk_ru_test, Y_test) == 1.0

In [15]:
precision_top1 = precision(uk_ru_test, mapping.predict(X_test), 1)
precision_top5 = precision(uk_ru_test, mapping.predict(X_test), 5)

assert precision_top1 >= 0.635
assert precision_top5 >= 0.813

### Улучшаем маппинг

Можно показать, что маппинг лучше строить ортогональным:
$$W^*= \arg\min_W ||WX - Y||_F \text{, где: } W^TW = I$$

Искать его можно через SVD:
$$X^TY=U\Sigma V^T\text{, singular value decompostion}$$

$$W^*=UV^T$$

**Задание** Реализуйте эту функцию.

In [16]:
def learn_transform(X_train, Y_train):
    """ 
    :returns: W* : float matrix[emb_dim x emb_dim] as defined in formulae above
    """
    U, _, V = np.linalg.svd(X_train.T @ Y_train)
    return U @ V

In [17]:
W = learn_transform(X_train, Y_train)

In [18]:
ru_emb.most_similar([np.matmul(uk_emb["серпень"], W)])

[('апрель', 0.8237907886505127),
 ('сентябрь', 0.8049712181091309),
 ('март', 0.8025652170181274),
 ('июнь', 0.8021842241287231),
 ('октябрь', 0.8001735806465149),
 ('ноябрь', 0.7934483289718628),
 ('февраль', 0.7914119958877563),
 ('июль', 0.790810763835907),
 ('август', 0.7891017198562622),
 ('декабрь', 0.7686372399330139)]

In [19]:
assert precision(uk_ru_test, np.matmul(X_test, W)) >= 0.653
assert precision(uk_ru_test, np.matmul(X_test, W), 5) >= 0.824

### Пишем переводчик

Реализуем простой пословный переводчик - для каждого слова будем искать его ближайшего соседа в общем пространстве эмбеддингов. Если слова нет в эмбеддингах - просто копируем его.

In [20]:
with open(fairy_tale_path, "r") as f:
    uk_sentences = [line.rstrip().lower() for line in f]

In [21]:
def translate(sentence):
    """
    :args:
        sentence - sentence in Ukrainian (str)
    :returns:
        translation - sentence in Russian (str)

    * find ukrainian embedding for each word in sentence
    * transform ukrainian embedding vector
    * find nearest russian word and replace
    """
    words = []
    for word in word_tokenize(sentence):
        if word in uk_emb.index_to_key:
            word = ru_emb.most_similar((uk_emb[word] @ W)[np.newaxis, ...])[0][0]
        words.append(word)
        
    return ' '.join(words)

In [22]:
assert translate(".") == "."
assert translate("1 , 3") == "1 , 3"
assert translate("кіт зловив мишу") == "кот поймал мышку"

In [23]:
for sentence in uk_sentences:
    print("src: {}\ndst: {}\n".format(sentence, translate(sentence)))

src: лисичка - сестричка і вовк - панібрат
dst: лисичка – сестричка и волк – панібрат

src: як була собі лисичка , да й пішла раз до однії баби добувать огню ; ввійшла у хату да й каже : " добрий день тобі , бабусю !
dst: как была себе лисичка , че и пошла раз к однії бабы добувать огня ; вошла во избу че и говорит : `` хороший день тебе , бабушку !

src: дай мені огня " .
dst: дай мне огня `` .

src: а баба тільки що вийняла із печі пирожок із маком , солодкий , да й положила , щоб він прохолов ; а лисичка се і підгледала , да тілько що баба нахилилась у піч , щоб достать огня , то лисичка зараз ухватила пирожок да і драла з хати , да , біжучи , весь мак із його виїла , а туда сміття наклала .
dst: а бабка только что вынула со печи пирожок со маком , сладкий , че и согнула , чтобы он прохолов ; а лисичка ой и підгледала , че токмо что бабка качнулась во печь , чтобы достать огня , то лисичка сейчас ухватила пирожок че и деру со хаты , че , пробежать , весь мак со его виїла , а туда му

src: побачивши , що вона їсть рибу , прибіг до їй да й каже : " здорово була , лисичко - сестричко !
dst: увидев , что она ест рыбу , прибежал к им че и говорит : `` здорово была , лисичко – сестричка !

src: де се ти набрала стільки риби ? "
dst: куда ой ты набрала столько рыбы ? ``

src: вона каже : " наловила , вовчику - братику ! "
dst: она говорит : `` наловила , вовчику – братику ! ``

src: а собі на думці : " подожди , і я зроблю з тобою таку штуку , як і ти зо мною " .
dst: а себе по мнении : `` подожди , и мной сделаю со тобой такую штуку , как и ты За мной `` .

src: - " як же ти ловила ? "
dst: – `` как то ты ловила ? ``

src: - " так , вовчику , уложила хвостик в ополонку , вожу тихенько да й кажу ; ловися , рибка , мала і велика !
dst: – `` так , вовчику , уложила хвостик во прорубь , вожу тихонько че и говорю ; ловися , рыбка , имела и большая !

src: коли хочеш , то і ти піди , налови собі " .
dst: когда хочешь , то и ты пойди , налови себе `` .

src: він побіг да зробив